<a href="https://colab.research.google.com/github/magdalenamorawska/Uczenie-Maszynowe-w-Finansach-/blob/TEST/zad10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark==1.3.0

In [ ]:
!pip install -q pyspark==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [ ]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

findspark.init("spark-3.5.1-bin-hadoop3")
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.appName('abc').getOrCreate()

In [ ]:
!free -mh # Komenda na sprawdzenie dostępnej pamięci RAM

               total        used        free      shared  buff/cache   available
Mem:            12Gi       926Mi       1.8Gi       1.0Mi         9Gi        11Gi
Swap:             0B          0B          0B


In [ ]:
sc.parallelize([1,2,3]).map(lambda x: x + 1).collect()

[2, 3, 4]

In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
   creating: data/shot_charts_top_10/
   creating: data/shot_charts_top_10/2013/
  inflating: data/shot_charts_top_10/2013/nba_savant201933.csv  
  inflating: data/shot_charts_top_10/2013/nba_savant2736.csv  
  inflating: data/shot_charts_top_10/2013/nba_savant101145.csv  
  inflating: data/shot_charts_top_10/2013/nba_savant200746.csv  
  inflating: data/shot_charts_top_10/2013/nba_savant200752.csv  
  inflating: data/shot_charts_top_10/2013/nba_savant201935.csv  
  inflating: data/shot_charts_top_10/2013/nba_savant101135.csv  
  inflating: data/shot_charts_top_10/2013/nba_savant202691.csv  
  inflating: data/shot_charts_top_10/2013/nba_savant201950.csv  
  inflating: data/shot_charts_top_10/2013/nba_savant202323.csv  
  inflating: data/shot_charts_top_10/2013/nba_savant201564.csv  
  inflating: data/shot_charts_top_10/2

Wgrywam dane i uzupelniam ewentualne braki

In [ ]:
df = spark.read.option('header','true')\
          .option('inferSchema','true')\
          .csv('data/season_totals.csv')

In [11]:
df.limit(5).show()

+---+-------------------+---+---+-------+---+----+----+---+----+------+---+----+-------+---+----+-------+-------+---+---+------+---+---+---+---+---+---+---+---+----+----+
|_c0|             player|pos|age|team_id|  g|  gs|  mp| fg| fga|fg_pct|fg3|fg3a|fg3_pct|fg2|fg2a|fg2_pct|efg_pct| ft|fta|ft_pct|orb|drb|trb|ast|stl|blk|tov| pf| pts|  yr|
+---+-------------------+---+---+-------+---+----+----+---+----+------+---+----+-------+---+----+-------+-------+---+---+------+---+---+---+---+---+---+---+---+----+----+
|  0|Abdul-Jabbar,Kareem|  C| 32|    LAL| 82|NULL|3143|835|1383| 0.604|  0|   1|    0.0|835|1382|  0.604|  0.604|364|476| 0.765|190|696|886|371| 81|280|297|216|2034|1980|
|  1|      Abernethy,Tom| PF| 25|    GSW| 67|NULL|1222|153| 318| 0.481|  0|   1|    0.0|153| 317|  0.483|  0.481| 56| 82| 0.683| 62|129|191| 87| 35| 12| 39|118| 362|1980|
|  2|        Adams,Alvan|  C| 25|    PHO| 75|NULL|2168|465| 875| 0.531|  0|   2|    0.0|465| 873|  0.533|  0.531|188|236| 0.797|158|451|609|322|1

In [13]:
data = df.select("yr", "fg", "fga", "player")
data = data.dropna()


In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


trenowanie modelu:
- przygotowanie danych
- podział na testowe i treningowe
- inizjalizacja i trenowanie modelu

In [15]:
assembler = VectorAssembler(inputCols=["yr"], outputCol="features")
data = assembler.transform(data)


In [16]:
train_data, test_data = data.randomSplit([0.8, 0.2])


In [17]:
lr = LinearRegression(featuresCol="features", labelCol="fga")
lr_model = lr.fit(train_data)

Predykcja:
- ewaluacja modelu, obliczenie dopasowania
-stworzenie dataframe dla roku 2022 i zaprognozowanie liczby rzutów

In [18]:
predictions = lr_model.transform(test_data)
predictions.select("yr", "fga", "prediction").show()

+----+---+------------------+
|  yr|fga|        prediction|
+----+---+------------------+
|1980|  3|476.85195613862015|
|1980| 11|476.85195613862015|
|1980| 13|476.85195613862015|
|1980| 24|476.85195613862015|
|1980| 25|476.85195613862015|
|1980| 33|476.85195613862015|
|1980| 37|476.85195613862015|
|1980| 64|476.85195613862015|
|1980| 50|476.85195613862015|
|1980| 69|476.85195613862015|
|1980| 56|476.85195613862015|
|1980| 63|476.85195613862015|
|1980| 65|476.85195613862015|
|1980|104|476.85195613862015|
|1980|105|476.85195613862015|
|1980| 79|476.85195613862015|
|1980|136|476.85195613862015|
|1980|146|476.85195613862015|
|1980|140|476.85195613862015|
|1980|137|476.85195613862015|
+----+---+------------------+
only showing top 20 rows



In [19]:
evaluator = RegressionEvaluator(labelCol="fga", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 374.3555313722813


In [20]:
data_2022 = spark.createDataFrame([(2022,)], ["yr"])
data_2022 = assembler.transform(data_2022)

prediction_2022 = lr_model.transform(data_2022)
prediction_2022.select("prediction").show()

+-----------------+
|       prediction|
+-----------------+
|323.2735443456286|
+-----------------+

